[![Wikidata logo](https://www.wikidata.org/static/images/project-logos/wikidatawiki.png)](https://www.wikidata.org)

# Wikidata API

This notebook is part of a Wikidata information & training event at the University of Zurich on 14.9.2017 ([more details here](https://www.wikidata.org/w/index.php?title=Wikidata:Events/Wikidata_Zurich)). Our goal is to help people interested in Wikidata, coding and open data in general, to get started with Wikidata as a source and platform for their projects. This workshop focuses on using the capabilities of the [MediaWiki API](https://www.mediawiki.org/wiki/API:Tutorial), as presented by [Cristina Sarasua](https://www.wikidata.org/wiki/User:Criscod) ([slides](https://docs.google.com/presentation/d/19eVu0IFdeB8gnSd7Nf0iLaPc0uh9lhIvgKUtp0Sghy0/edit#slide=id.g2545ae0d2e_0_22)), to extract entities and linked concepts from Wikidata.

Prepared by Oleg Lavrovsky ([Datalets.ch](https://datalets.ch)), facilitating on behalf of the [School of Data](https://schoolofdata.ch) working group at [Opendata.ch](https://opendata.ch). If you have any feedback or questions, please raise them directly to me or on [our forum](https://forum.schoolofdata.ch/t/wikidata-zurich-workshop-14-09/267/1).

[![School of Data logo](https://schoolofdata-ch.github.io/images/scoda-horizontal-212x60-nontransparent_en.png)](https://schoolofdata.ch)

## Prerequisites

We will be using the [Python programming language](https://python.org) for our demonstration, popular with academic, industrial and hobby/beginner users alike, it lends itself well to the task of connecting to Internet data sources and doing some analysis or visualisation. The example code here should run in any modern Python interpreter, available on most computing platforms (Windows, Mac, Linux, etc.). We particularly encourage you to try [Jupyter](https://jupyter.org), which makes working with this kind of Python project especially pleasant.

For participants of the workshop, we have set up a shared "Jupyter Hub" server, so you don't need to prepare anything except Internet connectivity and a reasonably modern web browser on your laptop. For those who wish to run set up their own environment in a short amount of time, check out the official [installation guide](http://jupyter.readthedocs.io/en/latest/install.html) which suggests the use of [Anaconda](https://www.continuum.io/downloads), or cloud-based offerings from [Sandstorm.io](https://apps.sandstorm.io/app/rprqf3t2h3vd3swfkhwk076qrennh9msreyjv70g7sgw5hkdxjph), [Google Cloud](https://cloud.google.com/solutions/using-tensorflow-jupyterhub-classrooms), [Microsoft Azure](https://notebooks.azure.com/) or [Domino Data Lab](https://www.dominodatalab.com/trial/).

## Goalposts

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/59/Chroicocephalus_ridibundus_in_Z%C3%BCrich_01.JPG/800px-Chroicocephalus_ridibundus_in_Z%C3%BCrich_01.JPG" width="300"><small>*Chroicocephalus ridibundus* by [Ximeg](https://commons.wikimedia.org/wiki/File:Chroicocephalus_ridibundus_in_Z%C3%BCrich_01.JPG), CC BY-SA 3.0</small></center>

A little bird lives in the big city of Zürich, thirsting for knowledge. He doesn't know how to read or to google, or even to ask a librarian for advice. Luckily for him, [Wikidata](https://www.wikidata.org) is here to help: by creating **structured data**, all of the collected knowledge of Wikipedia can be reused in infinite ways. Through the magic of APIs, this data can also be accessed in a variety of ways. Let's dig in and find out how!

## Under the hood

> Wikidata is available under a free license, exported using standard formats, and can be interlinked to other open data sets on the linked data web.

As mentioned in the [Introduction](https://www.wikidata.org/wiki/Wikidata:Introduction), Wikidata is central to the architecture of projects maintained by Wikimedia Foundation. It is connected to the system behind Wikipedia and its many sibling sites. The [API Sandbox](https://en.wikipedia.org/wiki/Special:ApiSandbox) along with the [tips here](https://www.mediawiki.org/wiki/API:Search_and_discovery) can help us to come up with some interesting queries. 

To be more precise and avoid the ambiguity of free-text search, it helps to use these facilities to define our interests in Wikidata by Items, identified by Q and a number, such as:

- [Zürich - Q72](https://www.wikidata.org/wiki/Q72)
- [Water - Q283](https://www.wikidata.org/wiki/Q283)
- [Fountain - Q483453](https://www.wikidata.org/wiki/Q483453)
- [Public art - Q557141](https://www.wikidata.org/wiki/Q557141)

Having some starting places in the tree of knowledge, we can run some [Wikidata API](https://www.wikidata.org/w/api.php) queries to get information about these subjects. MediaWiki's API is language-angostic, that is, it can be accessed using **any** programming language which supports the widespread [RESTful API](https://en.wikipedia.org/wiki/Representational_state_transfer) standard. You shouldn't need to know much about this unless you are a Web developer, and assume that it will probably "just work". 

Depending on the programming language, there may be libraries which give you a little more support. In Python, such APIs are often accessed using [Requests](http://docs.python-requests.org/en/master/). This comes standard in most installations and frameworks, or can be easily added to your system. The call to the Wikidata API to obtain details on our first item would then look like this: 

In [4]:
import requests # this loads the library we need

ITEM = "Q72" # identifier of the item we are looking for

# from the MediaWiki API documentation, a "query string":
URL = "https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&format=json"

r = requests.get(URL % ITEM) # here we make the actual call to the remote server
r.status_code # if it worked, the next line should read 200...

200

You can now look at the structure of the data by running the `r.json()` command. But we will skip right to the juicy details:

In [2]:
Zurich = r.json()['entities']['Q72']
Zurich['aliases']['en'][0]['value'] # should equal City of Zurich

'City of Zurich'

In [3]:
Zurich['descriptions']['en']['value']

'capital of the canton of Zürich, Switzerland'

In [9]:
from ipywidgets import Image

flagfile = Zurich['claims']['P41'][0]['mainsnak']['datavalue']['value'].replace(' ', '_')
flagurl = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/%s/200px-%s.png" % (flagfile, flagfile)
Image(value=requests.get(flagurl).content, width=100, height=100)

We can also try the example from Cristina's presentation, getting the contributions of a Wikipedia user:

In [11]:
USERID = "criscod"
URL = "https://www.wikidata.org/w/api.php?action=query&list=usercontribs&format=json&ucuser=%s&uclimit=5"

r = requests.get(URL % USERID)
for uc in r.json()['query']['usercontribs']:
    print(uc['title'] + " " + uc['comment'])

Wikidata:Events/Wikidata Zurich /* Workshop Materials */
Wikidata:Events/Wikidata Zurich /* Workshop Materials */
Wikidata:Events/Wikidata Zurich /*   HackZurich */
Wikidata:Events/Wikidata Zurich /*   Wikidata workshop */
Q38141077 /* wbsetlabel-add:1|es */ Campus de Oerlikon


## How libraries help

Dictionaries created from JSON are a popular way to work with data for developers, but for some people seeing so many brackets can be a maddening experience. Happily, there are nice people in the software community who create libraries that can help streamline working with APIs. 

The advantage is that you are working with (hopefully nicely documented) classes and functions. Libraries can help you tune your use of a web service for performance, and solve a lot of other headaches that other developers have run into. 

The main issue to think about it is that you are relying on an intermediary which may not keep up with the latest changes and new features of the API. Point in case.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Pwb_icon.svg/128px-Pwb_icon.svg.png)

A mature project called [Pywikibot](https://www.mediawiki.org/wiki/Manual:Pywikibot) was developed to do all kinds of useful things in Wikipedia, including automating queries. But it also packs in tons of functionality which we probably don't need, addressing other use cases. So let us try the much younger [Wikidata library](https://pypi.org/project/Wikidata/), which focuses specifically on providing access to the Wikidata API, and is being actively developed on [GitHub](https://github.com/dahlia/wikidata). 

To install it, we first run the `pip install wikidata` command in our project folder. Then we can do things like this:

In [5]:
from wikidata.client import Client
wikiclient = Client()
entity = wikiclient.get('Q72', load=True)
entity

<wikidata.entity.Entity Q72 'Zurich'>

If you didn't get any errors at this point, things get quite a bit easier. Take a look at the [wikidata library docs](https://wikidata.readthedocs.io/en/stable/) to help make sense of what's going on in these functions:

In [6]:
entity.description

m'capital of the canton of Zürich, Switzerland'

In [7]:
image_prop = wikiclient.get('P18')
image = entity[image_prop]
image.image_url

'https://upload.wikimedia.org/wikipedia/commons/3/3c/Zuerich_Fraumuenster_St_Peter.jpg'

In [10]:
Image(value=requests.get(image.image_url).content, width=100, height=100)

We can also combine the use of the `requests` and `wikidata` based API calls for slightly more complex queries, such as:

In [12]:
URL = "https://www.wikidata.org/w/api.php?action=wbgetclaims&entity=%s&property=%s&format=json"
q = URL % ('Q72', 'P31')
r = requests.get(q)
j = r.json()
r.status_code == 200

True

In [38]:
for c in j['claims']['P31']:
    QID = c['mainsnak']['datavalue']['value']['id']
    print(wikiclient.get(QID, load=True).description)

large and permanent human settlement
primary city of a political entity (country, state, county, etc)
capital or administrative center of a canton of Switzerland
community dominated by its university population
smallest government division in Switzerland
city with a population of more than 100,000 inhabitants


## Down to business

Yay! We were able to reproduce the basic use of the API. Now how about something a little more interesting for our feathered friend... 

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.